### Instalar y desinstalar cosas

In [1]:
!pip uninstall tensorflow -y

Found existing installation: tensorflow 2.16.1


Uninstalling tensorflow-2.16.1:


  Successfully uninstalled tensorflow-2.16.1


In [2]:
!pip install -q -U keras-nlp


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install -q keras==3.3.3


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


### Set up Keras JAX backend

In [4]:
import jax

jax.devices()

E0000 00:00:1725167507.056605      77 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
E0901 05:11:47.093010351      77 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2024-09-01T05:11:47.092992757+00:00", grpc_status:2}


[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [5]:
import os

# The Keras 3 distribution API is only implemented for the JAX backend for now
os.environ["KERAS_BACKEND"] = "jax"
# Pre-allocate 90% of TPU memory to minimize memory fragmentation and allocation
# overhead
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"

import keras
import keras_nlp

2024-09-01 05:11:59.238051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 05:11:59.266081: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 05:11:59.266125: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
keras.config.set_floatx("bfloat16")

## Load model

To load the model with the weights and tensors distributed across TPUs, first create a new `DeviceMesh`. `DeviceMesh` represents a collection of hardware devices configured for distributed computation and was introduced in Keras 3 as part of the unified distribution API.

The distribution API enables data and model parallelism, allowing for efficient scaling of deep learning models on multiple accelerators and hosts. It leverages the underlying framework (e.g. JAX) to distribute the program and tensors according to the sharding directives through a procedure called single program, multiple data (SPMD) expansion. Check out more details in the new [Keras 3 distribution API guide](https://keras.io/guides/distribution/).

In [7]:
# Create a device mesh with (1, 8) shape so that the weights are sharded across
# all 8 TPUs.
device_mesh = keras.distribution.DeviceMesh(
    (1, 8),
    ["batch", "model"],
    devices=keras.distribution.list_devices())

`LayoutMap` from the distribution API specifies how the weights and tensors should be sharded or replicated.

In [8]:
layout_map = keras_nlp.models.GemmaBackbone.get_layout_map(device_mesh)

`ModelParallel` allows you to shard model weights or activation tensors across all devcies on the `DeviceMesh`. In this case, some of the Gemma 2 27B model weights are sharded across 8 TPU cores according to the `layout_map` defined above.

In [9]:
model_parallel = keras.distribution.ModelParallel(
    device_mesh, layout_map, batch_dim_name="batch")

keras.distribution.set_distribution(model_parallel)



Now load the Gemma 2 27B model in the distributed way.

In [10]:
gemma2_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_9b_en")
gemma2_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 3584)        │   9,241,705,984 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     917,504,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 9,241,705,984 (17.21 GB)

 Trainable params: 9,241,705,984 (17.21 GB)

 Non-trainable params: 0 (0.00 B)

In [11]:
sampler = keras_nlp.samplers.RandomSampler()
gemma2_lm.compile(sampler=sampler)

In [12]:
import pandas as pd
import random

# Set the random seed for reproducibility
random.seed(42)

# Function to create a prompt with randomly ordered answers
def create_prompt_with_random_answers(article, question, correct_answer, wrong_answer):
    # Randomly decide whether to place the correct answer as 1 or 2
    if random.choice([True, False]):
        answer_1, answer_2 = correct_answer, wrong_answer
        correct_answer_position = 1
    else:
        answer_1, answer_2 = wrong_answer, correct_answer
        correct_answer_position = 2

    # Create the prompt string
    prompt = f"""<bos><start_of_turn>user
This is a passage:
---
{article}
---
Please analyze the passage carefully and select the correct answer for the following question. Before choosing your answer, explain your reasoning step by step:
---
<question>{question}<\question>
---
Answer options:
Option A1A. <answer>{answer_1}<\answer>
Option A2A. <answer>{answer_2}<\answer>
---
Write your reasoning step by step and THEN write your final answer.
Indicate your response as "The answer is A1A" if you chose option 1 or "The answer is A2A" if you chose option 2 based on the option you believe is correct.

Make sure to indicate your response as "The answer is A1A" or "The answer is A2A" in the correct format, otherwise it will be unreadable.<end_of_turn>
<start_of_turn>model"""
    
    return prompt, correct_answer_position

# Function to process a single CSV file and generate the prompts and correct answer positions
def process_dataset(csv_file):
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Lists to store the prompts and correct answer positions
    prompts = []
    correct_answer_positions = []

    # Iterate through the dataset to create prompts and track correct answer positions
    for _, row in df.iterrows():
        article = row['article']
        question = row['question']
        correct_answer = row['correct_answer']
        wrong_answer = row['wrong_answer']

        prompt, correct_answer_position = create_prompt_with_random_answers(article, question, correct_answer, wrong_answer)
        prompts.append(prompt)
        correct_answer_positions.append(correct_answer_position)

    return prompts, correct_answer_positions

# Process the easy and hard development datasets separately
prompts_easy, correct_answers_easy = process_dataset('/kaggle/input/quality-preprocessed/selected_data_easy_dev.csv')
prompts_hard, correct_answers_hard = process_dataset('/kaggle/input/quality-preprocessed/selected_data_hard_dev.csv')

# Example of how to print the results
# for i, prompt in enumerate(prompts_easy[:3]):  # Print first 3 prompts as an example for easy
#     print(f"Easy Prompt {i+1}:\n{prompt}\nCorrect answer position: {correct_answers_easy[i]}\n")

# for i, prompt in enumerate(prompts_hard[:3]):  # Print first 3 prompts as an example for hard
#     print(f"Hard Prompt {i+1}:\n{prompt}\nCorrect answer position: {correct_answers_hard[i]}\n")

In [13]:
import re

def extract_answer_number(generated_text):
    # Find the part of the text after "<start_of_turn>model"
    model_response = re.split(r'<start_of_turn>model', generated_text, maxsplit=1)[-1]

    match = re.search(r'A(\d)A', model_response[::-1])
    #print(match)
    if match:
        return int(match.group(1))
    else:
        return None  # Return None if no answer is found

In [14]:


# Example usage
generated_output = gemma2_lm.generate(prompts_easy[0])
#print(generated_output)
answer_number = extract_answer_number(generated_output)
print(f"Extracted answer number: {answer_number}")

Extracted answer number: 1


In [15]:
# Function to evaluate the model's performance on a given set of prompts
def evaluate_model(prompts, correct_answers, dataset_name):
    model_outputs = []
    extracted_answers = []

    # Generate outputs in parallel (in batches of 4)
    for i in range(0, len(prompts), 4):
        batch_prompts = prompts[i:i+4]
        generated_outputs = gemma2_lm.generate(batch_prompts)
        
        for output in generated_outputs:
            model_outputs.append(output)
            extracted_answer = extract_answer_number(output)
            extracted_answers.append(extracted_answer)
            print(extracted_answer)

    # Save results to CSV
    results_df = pd.DataFrame({
        'prompt': prompts,
        'model_output': model_outputs,
        'correct_answer': correct_answers,
        'extracted_answer': extracted_answers
    })
    results_df.to_csv(f'{dataset_name}_evaluation_results.csv', index=False)

# Test with the first 5 rows of the easy dataset
evaluate_model(prompts_easy[:5], correct_answers_easy[:5], "easy_dev_sample")


1
1
2
1


1


In [16]:
# Evaluate on the full datasets
evaluate_model(prompts_easy, correct_answers_easy, "easy_dev_full")
evaluate_model(prompts_hard, correct_answers_hard, "hard_dev_full")

1
1
2
1


1
1
1
1


2
1
1
1


1
1
1
1


2
1
2
2


...

2
1
2
1


2
2
2
1


1
1
1
2


1
1
2
1


2
